Important Note:


Important Note:
If you wish to run the training yourself, it's recommended to swap the Runtime to gpu. Do this by going to Runtime -> change runtime type


In [ ]:
!pip install chess
!pip install gdown

Imports

In [ ]:
import chess
import gdown
import numpy as np
from google.colab import files
import torch
import pandas as pd
from torch.optim.lr_scheduler import StepLR
from torch.optim import RMSprop
import time
from tqdm import tqdm

Download the data.csv file

In [ ]:
data_file_id = "1JEzNboVftrLKe1YptqIYqAu5T-IE76PH"
url = f'https://drive.google.com/uc?id={data_file_id}'
gdown.download(url, 'data.csv', quiet=False)

Download our trained model

In [ ]:
trained_model_file_id = "1GfSRtp9IOZ3mfExnWaRLZ35Fm2Bwe7rn"
url = f'https://drive.google.com/uc?id={trained_model_file_id}'
gdown.download(url, 'model.pth', quiet=False)

Piece encoding with perspective shift for black, and input builder function

In [ ]:
def calculate_index(sqr_index, piece_type, side, side_to_move) -> int:
    if side_to_move == chess.BLACK:
        side = 1 - side
        sqr_index ^= 0b111000

    return side * 64 * 6 + (piece_type - 1) * 64 + sqr_index


def fen_to_feature_vector(fen: str) -> np.ndarray:
    board = chess.Board(fen)
    vec = np.zeros(768, dtype=np.float32)
    for piece_type in range(1, 7):
        for color in [chess.WHITE, chess.BLACK]:
            bb = int(board.pieces(piece_type, color))
            while bb:
                sq = (bb & -bb).bit_length() - 1
                idx = calculate_index(sq, piece_type, int(color), int(board.turn))
                vec[idx] = 1.0
                bb &= bb - 1
    return vec



Read the data

In [ ]:
df = pd.read_csv('/content/data.csv')
print(df.shape)

Flip the evaluation if its blacks turn. Our network is always from the perspective of side to move.

In [ ]:
df['Evaluation'] = df.apply(lambda row: -row['Evaluation'] if row['FEN'].split()[1] == 'b' else row['Evaluation'], axis=1)

Split the data to train, validation, test

In [ ]:
split_train = 0.8
split_val = 0.1
split_test = 1 - split_train - split_val

n = df.shape[0]
m_train, m_val = int(n * split_train), int(n * split_val)
train_df = df[:m_train].copy()
val_df = df[m_train:m_train + m_val].copy()
test_df = df[m_train + m_val:].copy()

train_df.shape, val_df.shape, test_df.shape

Normilize our targets for smoother training. Training mean and std values will be used for denormilizing our predicition

In [ ]:
y_train = train_df['Evaluation'].values.astype(np.float32)
y_val =  val_df['Evaluation'].values.astype(np.float32)
y_test =  test_df['Evaluation'].values.astype(np.float32)
y_train_mean = y_train.mean()
y_train_std = y_train.std()
y_val_mean = y_val.mean()
y_val_std = y_val.std()
y_test_mean = y_test.mean()
y_test_std = y_test.std()

train_df['Evaluation'] = (train_df['Evaluation'] - y_train_mean) / y_train_std
val_df['Evaluation'] = (val_df['Evaluation'] - y_val_mean) / y_val_std
test_df['Evaluation'] = (test_df['Evaluation'] - y_test_mean) / y_test_std


Define a dataset class that will use our board encoder

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ChessFENDataset(Dataset):
    def __init__(self, fens, evals, encoder_fn):
        self.fens = fens
        self.evals = evals
        self.encoder = encoder_fn

    def __len__(self):
        return len(self.fens)

    def __getitem__(self, idx):
        x = torch.tensor(self.encoder(self.fens[idx]), dtype=torch.float32)
        y = torch.tensor(self.evals[idx], dtype=torch.float32)
        return x, y

Define the model. When playing, the forward function will make use of efficient updates, thus speeding up evaluation

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class NNUE(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 8)
        self.fc5 = nn.Linear(8, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = self.fc5(x)
        return x

Define data sets and loaders

In [ ]:
train_dataset = ChessFENDataset(train_df['FEN'].values, train_df['Evaluation'].values, fen_to_feature_vector)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

val_dataset = ChessFENDataset(val_df['FEN'].values, val_df['Evaluation'].values, fen_to_feature_vector)
val_loader = DataLoader(val_dataset, batch_size=512)

test_dataset = ChessFENDataset(test_df['FEN'].values, test_df['Evaluation'].values, fen_to_feature_vector)
test_loader = DataLoader(test_dataset, batch_size=512)

Swap to gpu if available. Please enable it in your noteobok by going to Runtime->change runtime type

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")

Create a new model for training (No need to do this if you're just testing)

In [ ]:
model = NNUE().to(device)

Or load our model for testing

In [ ]:
our_model = NNUE()
our_model.load_state_dict(torch.load('/content/model.pth', map_location=device))
our_model.to(device)

Define parameters for training.

In [ ]:

criterion = torch.nn.SmoothL1Loss()
optimizer = RMSprop(model.parameters(), lr=1e-4)
best_val_loss = float('inf')
patience = 5
patience_counter = 0
epochs = 40
scheduler = StepLR(optimizer, step_size=8, gamma=0.7)

**Train the new model**! We train with SmoothL1 loss but print MAE for visual indication (Will take a long time, skip to testing instead)

In [ ]:
print("Starting")
for epoch in range(epochs):
    model.train()
    train_losses = []
    train_mae_losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.unsqueeze(1).float().to(device)

        optimizer.zero_grad()
        preds = model(x_batch)

        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

        preds_unnorm = preds * y_train_std + y_train_mean
        y_unnorm = y_batch * y_train_std + y_train_mean
        mae_loss = torch.mean(torch.abs(preds_unnorm - y_unnorm))
        train_mae_losses.append(mae_loss)

    model.eval()
    val_losses = []
    val_mae_losses = []
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device).unsqueeze(1)
            preds = model(x_batch)


            loss = criterion(preds, y_batch)
            val_losses.append(loss.item())
            preds_unnorm = preds * y_train_std + y_train_mean
            y_unnorm = y_batch * y_val_std + y_val_mean
            mae_loss = torch.mean(torch.abs(preds_unnorm - y_unnorm))
            val_mae_losses.append(mae_loss)

    avg_train_loss = np.mean(train_losses)
    avg_val_loss = np.mean(val_losses)
    avg_train_mae = np.mean([x.detach().cpu().numpy() for x in train_mae_losses])
    avg_val_mae = np.mean([x.detach().cpu().numpy() for x in val_mae_losses])


    print(f"Epoch {epoch+1}: Train MAE = {avg_train_mae:.5f}, Val MAE = {avg_val_mae:.5f}, "
          f"Train Loss = {avg_train_loss:.5f}, Val Loss = {avg_val_loss:.5f}")
    scheduler.step()
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = model.state_dict()
        patience_counter = 0
        torch.save(best_model_state, '/content/best_model.pth')
        print(f"Best model saved at epoch {epoch+1}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

model.load_state_dict(best_model_state)

Test our model! Printed error is MAE in centipawn. (100 points = 1 pawn value) Result is ~59.

In [ ]:
our_model.eval()
test_mae_losses = []


with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device).unsqueeze(1).float()

        preds = our_model(x_batch)

        # Unnormalize predictions and targets
        preds_unnorm = preds * y_train_std + y_train_mean
        y_unnorm = y_batch * y_test_std + y_test_mean

        mae_loss = torch.mean(torch.abs(preds_unnorm - y_unnorm))
        test_mae_losses.append(mae_loss)

avg_test_mae = np.mean([x.detach().cpu().numpy() for x in test_mae_losses])
print(f"Test MAE = {avg_test_mae:.5f}")